# COVID screening lab simulation

## Requirements and module imports

Code in this simulation uses a standard Anaconda Python environment (https://www.anaconda.com/distribution/#download-section). Additionally this model uses SimPy3 (https://simpy.readthedocs.io/en/latest/). Install SimPy3 with `pip install 'simpy<4'`.

In [1]:
import simpy
import inspect
from sim_utils.replication import Replicator
from sim_utils.parameters import Scenario

## Set up scenarios

Parameters defined in scenarios will overwrite default values in the parameters python file.

In [2]:
# Set up a dictionary to hold scenarios
scenarios = {}

scenarios['30k_recommended'] = Scenario(
    
    samples_per_day = 30132,
    
    basic_batch_size = 93,

    resource_numbers = {
            'human_sample_receipt': 16,
            'human_sample_prep': 14,
            'human_rna_prep': 4,
            'human_pcr': 4,
            'sample_heat_block': 15,
            'beckman_rna_extraction': 17,
            'pcr_plate_stamper': 6,
            'pcr_plate_reader': 17,
            'sample_prep_automation': 3
            },
    
    # Workstations represent the physical capacity (max jobs)
    # 7 auto sample prep or 16 manual
    workstation_capacity =  {
            'workstation_0': 99999,
            'workstation_1a': 8,
            'workstation_1b_man': 9,
            'workstation_1b_auto': 3, 
            'workstation_1c': 15, 
            'workstation_2': 16,
            'workstation_3': 4,
            'workstation_4': 13
            },

    # Resource available hours (use hours)
    resource_shift_hours =  {
            'human_sample_receipt': (0.3, 17.75),
            'human_sample_prep': (0.3, 17.75),
            'human_rna_prep': (0.3, 17.75),
            'human_pcr': (0.3, 17.75),
            'sample_heat_block': (0.0, 24.0),
            'beckman_rna_extraction': (0.0, 24.0),
            'pcr_plate_stamper': (0.0, 24.0),
            'pcr_plate_reader': (0.0, 24.0),
            'sample_prep_automation': (0.0, 24.0),
            },
    
    # FTE should always be present for start times
    process_start_hours = {
            'sample_receipt': (0., 15.5),
            'sample_prep': (0., 15.5),
            'sample_heat': (0., 15.5),
            'rna_extraction': (0., 24),
            'pcr_prep': (0., 24),
            'pcr': (0.3, 24)
            },
    
    # Process duration. Tuple of fixed time, time per entity, and time per item in entity.
    # Multi-step automated processes have three sets of times (set up, automated, clean down)
    process_duration = {
             'batch_input': ([0,0,0],),
             'sample_receipt': ([16, 0, 0],),
             'sample_prep_manual': ([37, 0, 0],),
             'sample_prep_auto': ([2, 0, 0], [6, 0, 0], [8, 0, 0]),
             'sample_heat':  ([1, 0, 0], [20, 0, 0], [2, 0, 0]),
             'rna_extraction': ([15,0,0],[77,0,0],[5,0,0]),
             'pcr_prep': ([31,0,0],[10,0,0],[1,0,0]),
             'pcr': ([5,0,0],[117,0,0],[1,0,0]),
             'data_analysis': ([0,0,0],)
             },
    
    # Resource unavailability on any whole day due to breakdown
    resource_breakdown_unavailability = {
            'human_sample_receipt': 0,
            'human_sample_prep': 0,
            'human_rna_prep': 0,
            'human_pcr': 0,
            'sample_heat_block': 0,
            'beckman_rna_extraction': 0.04,
            'pcr_plate_stamper': 0.08,
            'pcr_plate_reader': 0.02,
            'sample_prep_automation': 0
            },
    
              
    allow_maual_sample_prep = True,
    tea_break_times = [2*60, 7*60, 12*60, 16*60],
    meal_break_times = [4*60, 14*60],
    break_start_spread = 60,
    tea_break_duration = [0,0],
    meal_break_duration = [0,0]
    )

## Run model

In [3]:
replications = 30
replications = Replicator(scenarios, replications)
replications.run_scenarios()

                                                                               
Output results
--------------
                                                min          mean    median  \
                                             Result        Result    Result   
result_type               name                                                
demand_met                30k_recommended      0.89      0.994000      1.00   
input                     30k_recommended  30132.00  30132.000000  30132.00   
max_process_time_hours    30k_recommended     22.73     23.156667     22.92   
median_process_time_hours 30k_recommended     22.73     23.156667     22.92   
output                    30k_recommended  26784.00  29946.000000  30132.00   

                                                max  
                                             Result  
result_type               name                       
demand_met                30k_recommended      1.00  
input                     30k_recommended

## Show model default parameters

Run the code below to model defaults these are over-ridden by scenarios).

In [4]:
print(inspect.getsource(Scenario.__init__))

    def __init__(self, *initial_data, **kwargs):
        
        # Set default values
        # 16/4/2020 Adjust paramters so that day starts with FTE arrival


        # Work arrival
        self.samples_per_day = 30000
        self.deliveries_per_day = 1
        self.basic_batch_size = 93

        # Day and run parameters
        # 16/4/2020 Model is designed to run primarily in single days
        self.day_duration = 1440
        self.run_days = 1
        self.warm_up_days = 0
        
        # Breaks for people (high prority job, but does not interupt work)
        # Times from start of FTE day (6am)
        self.tea_break_times = [2*60, 7*60, 12*60, 16*60]
        self.meal_break_times = [4*60, 14*60]
        # Spread start of break for people randomly after set start times
        self.break_start_spread = 60
                
        # break duration is a uniform distribution between min and max
        self.tea_break_duration = [25, 35]
        self.meal_break_duration = [40, 